# Amazon SageMaker model bias monitor: Fairness and explainability with SageMaker Clarify

Amazon SageMaker Clarify는 잠재적 편향을 감지하고 이러한 모델이 예측을 수행하는 방법을 설명하여 기계 학습 모델을 개선하는 데 도움이 됩니다. Clarify에서 제공하는 공정성 및 설명 가능성 기능은 신뢰할 수 있고 이해하기 쉬운 기계 학습 모델을 구축하는 데 한 걸음 더 다가갑니다. 제품에는 다음 작업에 도움이 되는 도구가 함께 제공됩니다.

- 머신 러닝(ML) 수명 주기의 각 단계에서 발생할 수 있는 편향을 측정합니다(데이터 수집, 모델 교육 및 조정, 추론을 위해 배포된 ML 모델 모니터링).
- 위험 및 규정 준수 팀과 외부 규제 기관을 대상으로 하는 모델 거버넌스 보고서를 생성합니다.
- 예측을 평가하는 데 사용되는 데이터, 모델 및 모니터링에 대한 설명을 제공합니다.

이 실습에서는 프로덕션 모델과 SageMaker 엔드포인트가 이미 배포되어 있습니다. SageMaker Clarify를 사용하여 훈련 데이터 세트 및 검증 데이터 세트를 분석하고 데이터 세트의 사전 훈련 편향과 모델의 사후 훈련 편향을 측정합니다. 마지막으로 모델 결정에 대한 다양한 입력 기능의 중요성에 대한 설명 가능성 보고서를 생성하고 검토합니다.


![MLOPS Amazon Clarify architecture](images/ClarifySDK.png)



## 가져오기 및 구성

다음 셀을 실행하여 필요한 Python 모듈을 가져옵니다.

In [4]:
import os
import copy
import json
import random
import time
import datetime
import boto3
import pandas as pd

from datetime import datetime, timedelta

from sagemaker import get_execution_role, image_uris, Session
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.predictor import Predictor

from sagemaker.model_monitor import (
    BiasAnalysisConfig,
    CronExpressionGenerator,
    DataCaptureConfig,
    EndpointInput,
    ExplainabilityAnalysisConfig,
    ModelBiasMonitor,
    ModelExplainabilityMonitor,
)

from sagemaker.clarify import (
    BiasConfig,
    DataConfig,
    ModelConfig,
    ModelPredictedLabelConfig,
    SHAPConfig,
)

다음 코드를 실행하여 나중에 사용할 modelArtifactBucket의 전체 이름을 확인합니다.

In [5]:
bucket = ''
s3 = boto3.resource('s3')
for buckets in s3.buckets.all():
    if 'modelartifactbucket' in buckets.name:
        bucket = buckets.name

print(f"Bucket Name: {bucket}")

Bucket Name: qls-179916-d8b842bf335ab6f7-modelartifactbucket-wpfssgx2i4mp


다음 코드를 실행하여 SageMaker 프로덕션 엔드포인트 이름 및 프로덕션 모델 이름을 미리 입력합니다.

In [6]:
# Get production endpoint name
endpoint_name = boto3.Session().client('sagemaker').list_endpoints(SortBy='CreationTime')['Endpoints'][0]['EndpointName']

# Get production model name
model_name = boto3.Session().client('sagemaker').list_models(SortBy='CreationTime')['Models'][0]['ModelName']

# Check if the endpoint is in service
client = boto3.client('sagemaker')
result = False
while result is False:
    response = client.describe_endpoint(EndpointName=endpoint_name)
    if (response['EndpointStatus'] == 'InService'):
        print(f"Production Endpoint Name:{endpoint_name}")
        print(f"Production Model Name:{model_name}")
        result = True
    else:
        print("Waiting for endpoint to be active")
        time.sleep(5)
        result = False

Production Endpoint Name:ProductionEndpoint-2qDGQ8QJTT2X
Production Model Name:ProductionModel-ZuQ2zlP5q8Ke


다음 코드를 실행하여 필요한 IAM 역할 및 AWS 리전을 가져옵니다.

In [7]:
role = get_execution_role()
print(f"RoleArn: {role}")

sagemaker_session = Session()
sagemaker_client = sagemaker_session.sagemaker_client
sagemaker_runtime_client = sagemaker_session.sagemaker_runtime_client

region = sagemaker_session.boto_region_name
print(f"AWS region: {region}")

RoleArn: arn:aws:iam::916295725490:role/qls-179916-d8b842bf335ab6f7-SageMakerStudioRole-1JR7QZ6UN5M4J
AWS region: us-west-2


다음 코드를 실행하여 캡처된 데이터 및 보고서를 저장하기 위한 Amazon S3 경로를 생성합니다.

In [8]:
prefix = 'sagemaker'
s3_key = f"s3://{bucket}/{prefix}"
print(f"S3 key: {s3_key}")

code_prefix = '{}/code'.format(prefix)
data_capture_prefix = f'{prefix}/datacapture'
s3_capture_upload_path = f'{s3_key}/datacapture'
s3_report_path = f'{s3_key}/reports'

print(f"Capture path: {s3_capture_upload_path}")
print(f"Report path: {s3_report_path}")


S3 key: s3://qls-179916-d8b842bf335ab6f7-modelartifactbucket-wpfssgx2i4mp/sagemaker
Capture path: s3://qls-179916-d8b842bf335ab6f7-modelartifactbucket-wpfssgx2i4mp/sagemaker/datacapture
Report path: s3://qls-179916-d8b842bf335ab6f7-modelartifactbucket-wpfssgx2i4mp/sagemaker/reports


다음 코드를 실행하여 기차를 읽고 데이터 파일의 유효성을 검사합니다.

In [9]:
test_dataset = "data/validate.csv"
train_dataset = "data/train.csv"
dataset_type = "text/csv"

with open(train_dataset) as f:
    headers_line = f.readline().rstrip()
all_headers = headers_line.split(",")
label_header = all_headers[0]

## 샘플 트래픽으로 엔드포인트 테스트

다음 코드를 실행하여 데이터 세트 유효성 검사에서 샘플의 하위 집합을 추출하고 샘플 데이터를 로컬 csv 파일로 씁니다. 두 개의 파일 세트가 있습니다.

- data-test.csv는 기능 데이터만 포함합니다.
- data-test-label.csv는 레이블 및 기능 데이터를 포함합니다.

In [10]:
# Here we are randomly picking subset of data from test datasets.
import itertools

shape = pd.read_csv(test_dataset)

a = [10*i for i in range(3)]
b = [10+i for i in range(10)]
indices = [i+j for i,j in itertools.product(a,b)]

test_data = shape.drop(shape.columns[[0]],axis=1)
test_data = test_data.iloc[indices]
test_data_with_label = shape.iloc[indices]

In [11]:
test_data.to_csv("data/data-test.csv",index=False,header=False)
test_data_with_label.to_csv("data/data-test-label.csv",index=False,header=False)

다음 코드를 실행하여 예측을 수행합니다. 이전 단계에서 채운 샘플 데이터 세트를 사용합니다. 실행이 끝나면 예측이 "0" 또는 "1"로 표시됩니다.

In [12]:
# Invoke Endpoint with test data for a quick test
print(f"Sending test traffic to the endpoint {endpoint_name}. \nPlease wait...")
predictions = ''

with open('data/data-test.csv', 'r') as f:
    for row in f:
        print(".", end="", flush=True)
        payload = row.rstrip('\n')
        response = sagemaker_runtime_client.invoke_endpoint(EndpointName=endpoint_name,
                                   ContentType="text/csv",
                                   Body=payload)
        predictions = ','.join([predictions, response['Body'].read().decode('utf-8')])
        time.sleep(0.5)

predictions = predictions.replace('\n','')
predictions = predictions.split(",")
predictions.pop(0)
print("="*20)
print(predictions)
print("Done!") 

Sending test traffic to the endpoint ProductionEndpoint-2qDGQ8QJTT2X. 
Please wait...
..............................====================
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
Done!


## 캡처된 데이터 보기

다음 코드를 실행하여 S3에 저장된 데이터 캡처 파일을 나열합니다. 호출이 발생한 시간을 기준으로 구성된 다른 기간의 다른 파일이 표시될 것으로 예상해야 합니다. 출력에 실패 로그가 표시되면 셀을 다시 실행하십시오.


In [13]:
print("Waiting 2 minutes for file to load...")
time.sleep(120)
s3_client = boto3.Session().client('s3')
current_endpoint_capture_prefix = '{}/{}'.format(data_capture_prefix, endpoint_name)
result = s3_client.list_objects(Bucket=bucket, Prefix=current_endpoint_capture_prefix)
capture_files = [capture_file.get("Key") for capture_file in result.get('Contents')]
print("Found Capture Files:")
print("\ns3://"+bucket+str(capture_files[0]))

Waiting 2 minutes for file to load...
Found Capture Files:

s3://qls-179916-d8b842bf335ab6f7-modelartifactbucket-wpfssgx2i4mpsagemaker/datacapture/ProductionEndpoint-2qDGQ8QJTT2X/ProductionModel-ZuQ2zlP5q8Ke/2021/10/10/02/00-27-030-7abd8940-5270-42b4-a575-038f1afa1124.jsonl


단일 캡처 파일의 내용을 보려면 다음 코드를 실행하십시오. 여기에서 SageMaker 특정 JSON 형식 파일에 캡처된 모든 데이터가 표시되어야 합니다. 캡처된 파일의 처음 몇 줄을 빠르게 살펴보십시오.

In [14]:
def get_obj_body(obj_key):
    return s3_client.get_object(Bucket=bucket, Key=obj_key).get('Body').read().decode("utf-8")

capture_file = get_obj_body(capture_files[-1])
print(capture_file[:2000])

{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"105,0,193.7,108,183.2,124,293.7,72,10.8,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0","encoding":"CSV"},"endpointOutput":{"observedContentType":"text/csv; charset=utf-8","mode":"OUTPUT","data":"0\n","encoding":"CSV"}},"eventMetadata":{"eventId":"57938e80-3d90-4d7b-8d2b-6be079d971c9","inferenceTime":"2021-10-10T02:00:27Z"},"eventVersion":"0"}
{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"114,4,141.3,96,230.4,88,223.7,85,9.4,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1","encoding":"CSV"},"endpointOutput":{"observedContentType":"text/csv; charset=utf-8","mode":"OUTPUT","data":"0\n","encoding":"CSV"}},"eventMetadata":{"eventId":"2cbe6e7d-9826-450d-9a1d-451ce925c993","inferenceTime":"2021-10-10T02:00:27Z"},"eventV

다음 코드를 실행하여 형식이 지정된 JSON 객체에서 한 줄을 살펴 봅니다.

In [15]:
import json
print(json.dumps(json.loads(capture_file.split('\n')[0]), indent=2))

{
  "captureData": {
    "endpointInput": {
      "observedContentType": "text/csv",
      "mode": "INPUT",
      "data": "105,0,193.7,108,183.2,124,293.7,72,10.8,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0",
      "encoding": "CSV"
    },
    "endpointOutput": {
      "observedContentType": "text/csv; charset=utf-8",
      "mode": "OUTPUT",
      "data": "0\n",
      "encoding": "CSV"
    }
  },
  "eventMetadata": {
    "eventId": "57938e80-3d90-4d7b-8d2b-6be079d971c9",
    "inferenceTime": "2021-10-10T02:00:27Z"
  },
  "eventVersion": "0"
}


## Amazon SageMaker Clarify 설정

**참고:** 다음 코드 샘플들은 [SageMaker 샘플 코드](https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker_processing/fairness_and_explainability/fairness_and_explainability.ipynb) 의 일부이며 랩 요구사항에 맞춰 업데이트 되었습니다.

SageMaker Clarify 프로세서를 사용하려면 다음 코드를 실행하십시오.

Clarify 프로세서는 분산 클러스터에서 편향 감지를 실행하는 데 사용할 인프라를 설정하고 있습니다. 이 예에서는 클러스터에 하나의 인스턴스만 있습니다. Clarify 구성에 대한 자세한 내용은 [공정성과 설명성을 위한 Amazon SageMaker Clarify 처리 작업 구성](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-configure-processing-jobs.html)을 참조하십시오. .

In [16]:
from sagemaker import clarify
clarify_processor = clarify.SageMakerClarifyProcessor(role=role,
                                                      instance_count=1,
                                                      instance_type='ml.m5.xlarge',
                                                      sagemaker_session=sagemaker_session)

다음 코드를 실행하여 테스트 및 훈련 데이터 세트를 S3에 업로드합니다. _train_uri_ 및 _test_uri_는 다음 단계에서 dataconfig 및 modelconfig를 작성하여 S3에서 직접 데이터를 가져오는 데 사용됩니다.

In [17]:
from sagemaker.s3 import S3Uploader
from sagemaker.inputs import TrainingInput

train_uri = S3Uploader.upload('data/train.csv', 's3://{}/{}'.format(bucket, prefix))
train_input = TrainingInput(train_uri, content_type='csv')
test_uri = S3Uploader.upload('data/validate.csv', 's3://{}/{}'.format(bucket, prefix))

## 편향 감지

#### SageMaker Clarify는 다양한 메트릭을 사용하여 가능한 사전 및 사후 교육 편향을 감지하는 데 도움이 됩니다.

### DataConfig 및 ModelConfig 작성

**DataConfig** 개체는 데이터 I/O에 대한 몇 가지 기본 정보를 Clarify에 전달합니다. 입력 데이터 세트를 찾을 위치, 출력을 저장할 위치, 대상 열(레이블), 헤더 이름 및 데이터 세트 유형을 지정합니다.


Run the following code to define DataConfig.

In [18]:
bias_report_output_path = 's3://{}/{}/clarify-bias'.format(bucket, prefix)
bias_data_config = clarify.DataConfig(s3_data_input_path=train_uri,
                                      s3_output_path=bias_report_output_path,
                                      label='Churn',
                                      headers=all_headers,
                                      dataset_type='text/csv')

**ModelConfig** 개체는 학습된 모델에 대한 정보를 전달합니다. 프로덕션 모델에 대한 추가 트래픽을 방지하기 위해 Clarify는 처리 시 전용 엔드포인트를 설정하고 해제합니다.

- **instance_type** 및 **instance_count**는 Clarify에서 처리하는 동안 모델을 실행하는 데 사용되는 선호하는 인스턴스 유형과 인스턴스 수를 지정합니다. 테스트 데이터 세트는 작기 때문에 단일 표준 인스턴스로 이 예제를 실행하기에 충분합니다. 크고 복잡한 데이터 세트가 있는 경우 더 나은 인스턴스 유형을 사용하여 속도를 높이거나 더 많은 인스턴스를 추가하여 Spark 병렬화를 활성화할 수 있습니다.

- **accept_type**은 엔드포인트 응답 페이로드 형식을 나타내고, **content_type**은 엔드포인트에 대한 요청의 페이로드 형식을 나타냅니다.

다음 코드를 실행하여 ModelConfig를 정의합니다.

In [19]:
model_config = clarify.ModelConfig(model_name=model_name,
                                   instance_type='ml.m5.xlarge',
                                   instance_count=1,
                                   accept_type='text/csv',
                                   content_type='text/csv')

**ModelPredictedLabelConfig**는 예측 형식에 대한 정보를 제공합니다. 이 문제에 대해 이진 분류를 사용하고 있습니다. 랜덤 포레스트 모델은 샘플의 확률을 제공하므로 Clarify는 끝점을 호출한 다음 확률 임계값을 사용하여 편향 분석을 위해 확률을 이진 레이블로 변환합니다. 임계값 이상의 예측은 레이블 값 1로 해석됩니다. 임계값 이하인 경우 레이블 값 0으로 해석됩니다.

다음 코드를 실행하여 ModelPredictedLabelConfig를 정의합니다.

In [20]:
predictions_config = clarify.ModelPredictedLabelConfig(probability_threshold=0.8)

### BiasConfig 작성
Clarify는 또한 민감한 열(facets)이 무엇인지, 민감한 기능(facet_values_or_threshold)이 무엇인지, 바람직한 결과(label_values_or_threshold)가 무엇인지에 대한 정보가 필요합니다. Clarify는 facet_values_or_threshold와 label_values_or_threshold에 대한 범주형 데이터와 연속형 데이터를 모두 처리할 수 있습니다. 이 경우 범주형 데이터를 사용하고 있습니다.

이 정보는 BiasConfig API에 지정됩니다. 여기에서 긍정적인 결과는 Churn = 0, _Account Length_는 민감한 범주, _Day Calls_ 응답자는 민감한 그룹, _group_name_은 측정을 위한 하위 그룹을 형성하는 데 사용됩니다.

In [ ]:
bias_config = clarify.BiasConfig(label_values_or_threshold=[0],
                                facet_name='Account Length',
                                facet_values_or_threshold=[100],
                                group_name='Day Calls')

#### **사전 훈련 편향**

모델 훈련이 발생하기 전에 데이터에 편향이 있을 수 있습니다. 훈련을 시작하기 전에 데이터에 편향이 있는지 검사하면 데이터 수집 격차를 감지하고 기능 엔지니어링에 정보를 제공하며 데이터가 반영할 수 있는 사회적 편향을 이해하는 데 도움이 될 수 있습니다.

사전 훈련 편향 메트릭을 계산하는 데는 훈련된 모델이 필요하지 않습니다.

#### **교육 후 편향**

훈련 후 편향 메트릭을 계산하려면 훈련된 모델이 필요합니다.

편향되지 않은 훈련 데이터(편향 메트릭에 의해 측정된 공정성의 개념에 의해 결정됨)는 훈련 후 여전히 편향된 모델 예측을 초래할 수 있습니다. 이것이 발생하는지 여부는 하이퍼파라미터 선택을 포함한 여러 요인에 따라 다릅니다.

다음 실행과 같이 이러한 옵션을 run_pre_training_bias() 및 run_post_training_bias()로 별도로 실행하거나 run_bias()와 동시에 실행할 수 있습니다.

평가를 위해 다음 셀을 실행하여 Clarify 프로세서를 시작합니다. Clarify 실행을 완료하는 데 일반적으로 10분이 걸립니다. Clarify는 계산을 위한 임시 SageMaker 엔드포인트를 생성하고 계산이 완료되면 리소스를 삭제합니다.

In [ ]:
clarify_processor.run_bias(data_config=bias_data_config,
                           bias_config=bias_config,
                           model_config=model_config,
                           model_predicted_label_config=predictions_config,
                           pre_training_methods='all',
                           post_training_methods='all')

이제 실행이 완료되었으므로 편향 보고서를 검토해 보겠습니다.

#### **편향 보고서 보기**

Studio의 **Experiments** 탭에서 결과를 볼 수 있습니다.

1. 왼쪽 창에서 **SageMaker components and registries**를 선택합니다.

1. 드롭다운 메뉴에서 **Experimentals and trials**을 선택합니다.

1. **Unassigned trial components**를 더블클릭하여 엽니다.

1. 이름에 **clarify-bias**가 포함된 시험을 더블클릭하여 엽니다.

  편향 보고서가 새 탭에서 열립니다. 검토한 후 나머지 지침을 보려면 이 탭(*model_bias_clarify_ko_kr.ipynb* 탭)으로 돌아가야 합니다.

1. 열리는 새 탭에서 **bias report**를 선택하여 보고서를 검토합니다.

<img src="./recordings/bias_report.gif">

각 편향 메트릭에는 탐색할 수 있는 예제와 함께 자세한 설명이 있습니다. 다음은 검토할 몇 가지 측정항목입니다.

- **Class Imbalance(CI):** 유리한 그룹이 불리한 그룹보다 상당히 높은 비율로 데이터 세트에 표시되는지 또는 그 반대인지 감지합니다.
- **Total Variation Distance (TVD):** 레이블 분포의 이 측정값은 유리한 계층 레이블의 확률 분포와 불리한 계층의 확률 분포 사이의 해밍 거리(Hamming distance)의 절반입니다.
- **Conditional Demographic Disparity in Predicted Labels (CDDPL):** 이 메트릭은 모델이 동일한 클래스에 대해 승인된 결과의 비율보다 불리한 계층에 대한 거부된 결과의 더 큰 비율을 예측했는지 여부를 조사합니다.

<img src="./recordings/bias_detail.gif">

결과를 편리한 표에 요약할 수도 있습니다. 페이지 오른쪽에 있는 **table** 아이콘을 선택합니다.

<img src="./recordings/bias_report_chart.gif">




다음 S3 버킷에서 편향 보고서(pdf, html 및 ipynb 파일 형식)를 볼 수 있습니다.

In [ ]:
bias_report_output_path

## 예측 설명

모델이 결정을 내린 이유에 대한 설명이 필요한 비즈니스 요구 사항과 입법 규정이 확대되고 있습니다. SageMaker Clarify는 SHAP(SHapley Additive exPlanations)를 사용하여 각 입력 기능이 최종 결정에 미치는 영향을 설명합니다. SHAP는 각 데이터 인스턴스에 대해 예측된 출력에 대한 특성 값의 개별 기여도를 분석하고 양수 또는 음수 값으로 나타냅니다.

커널 SHAP 알고리즘에는 기준선(배경 데이터 세트라고도 함)이 필요합니다. Baseline 데이터셋 유형은 DataConfig의 dataset_type과 동일해야 하며, 베이스라인 샘플은 기능만 포함해야 합니다. 정의에 따라 기준 데이터 세트는 기준 데이터 세트 파일에 대한 S3 URI이거나 샘플의 현재 위치 목록이어야 합니다. 이 경우 후자를 선택하고 테스트 데이터 세트의 첫 번째 샘플을 제자리 목록에 넣습니다.

다음 코드를 실행하여 SHAP 구성을 추가합니다.

In [ ]:
shap_config = clarify.SHAPConfig(baseline=[test_data.iloc[0].values.tolist()],
                                 num_samples=15,
                                 agg_method='mean_abs',
                                 save_local_shap_values=False)

explainability_output_path = 's3://{}/{}/clarify-explainability'.format(bucket, prefix)
explainability_data_config = clarify.DataConfig(s3_data_input_path=train_uri,
                                s3_output_path=explainability_output_path,
                                label='Churn',
                                headers=all_headers,
                                dataset_type='text/csv')

다음 셀을 실행하여 Clarify를 시작하고 설명 가능성을 평가하십시오. 일반적으로 작업을 완료하는 데 10분이 걸립니다.

프로세스가 완료되기를 기다리는 동안 [공정성 및 설명성 명확화](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-fairness-and-explainability.html)에 대해 자세히 알아볼 수 있습니다.

In [ ]:
clarify_processor.run_explainability(data_config=explainability_data_config,
                                     model_config=model_config,
                                     explainability_config=shap_config)

이제 실행이 완료되었으므로 설명 가능성 보고서를 검토해 보겠습니다.

#### 설명 보고서 보기
편향 보고서와 마찬가지로 SageMaker Studio의 **실험** 탭에서 설명 보고서를 볼 수 있습니다.

1. 왼쪽 창에서 **SageMaker components and registries**를 선택합니다.

1. 드롭다운 메뉴에서 **Experimentals and trials**을 선택합니다.

1. **Unassigned trial components**를 선택합니다.

1. **clarify-explainability**이 포함된 이름을 선택하고 **model explainability**을 선택하여 보고서를 검토합니다.


<img src="./recordings/explainability_detail.gif">

#### **질문:** 보고서에 따르면 이탈 예측을 결정하는 데 가장 중요한 기능은 무엇입니까?

다음 S3 버킷에서 편향 보고서(PDF, HTML 또는 ipynb 형식)를 볼 수 있습니다.

In [ ]:
explainability_output_path

### 정리

마지막으로 이 데모에서 설정하고 사용한 리소스를 정리하는 것을 잊지 마십시오.
다음 S3 버킷에서 편향 보고서(PDF, HTML 또는 ipynb 형식)를 볼 수 있습니다.

In [ ]:
sagemaker_session.delete_model(model_name)